In [1]:
import pandas as pd
import unicodedata
import requests

# #send to rest API
import json 
import requests
from requests.auth import HTTPBasicAuth

In [2]:
under_rbi_rate=pd.read_csv('undernew.csv')
over_rbi_rate=pd.read_csv('overnew.csv')
def_rank = pd.read_csv('../scrapers/def_scraper/batter_rbi_def_rank.csv')
batter_team_2025 = pd.read_csv('../general/stats/batter_team_2025.csv')

In [3]:
def_rank.rename(columns={'Rank': 'def_rank'}, inplace=True)

In [4]:
def_rank.rename(columns={'Team': 'team'}, inplace=True)

In [5]:
batter_team_2025.rename(columns={'Team': 'team'}, inplace=True)

In [6]:
batter_team_2025.rename(columns={'Player': 'player'}, inplace=True)

In [7]:
team_mapping_5 = {
    'CHC': 'CHC',
    'LAD': 'LAD',
    'ARI': 'ARI',
    'PIT': 'PIT',
    'MIA': 'MIA',
    'TOR': 'TOR',
    'BOS': 'BOS',
    'ATH': 'ATH',
    'BAL': 'BAL',
    'SEA': 'SEA',
    'SDP': 'SDP',
    'ATL': 'ATL',
    'PHI': 'PHI',
    'TEX': 'TEX',
    'MIL': 'MIL',
    'WSN': 'WSH',
    'NYY': 'NYY',
    'STL': 'STL',
    'TBR': 'TBR',
    'DET': 'DET',
    'KCR': 'KCR',
    'CLE': 'CLE',
    'COL': 'COL',
    'NYM': 'NYM',
    'CIN': 'CIN',
    'MIN': 'MIN',
    'SFG': 'SFG',
    'HOU': 'HOU',
    'CHW': 'CHW',
    'LAAs': 'LAA'
}

batter_team_2025['team'] = batter_team_2025['team'].map(team_mapping_5)

In [8]:
team_mapping = {
    'Milwaukee Brewers': 'MIL',
    'Minnesota Twins': 'MIN',
    'Chicago Cubs': 'CHC',
    'Colorado Rockies': 'COL',
    'Washington Nationals': 'WSH',
    'Toronto Blue Jays': 'TOR',
    'Atlanta Braves': 'ATL',
    'Baltimore Orioles': 'BAL',
    'New York Yankees': 'NYY',
    'Philadelphia Phillies': 'PHI',
    'Pittsburgh Pirates': 'PIT',
    'Kansas City Royals': 'KCR',
    'Arizona Diamondbacks': 'ARI',
    'Boston Red Sox': 'BOS',
    'Detroit Tigers': 'DET',
    'Cleveland Guardians': 'CLE',
    'Tampa Bay Rays': 'TBR',
    'San Francisco Giants': 'SFG',
    'Los Angeles Dodgers': 'LAD',
    'Seattle Mariners': 'SEA',
    'Cincinnati Reds': 'CIN',
    'Texas Rangers': 'TEX',
    'Chicago White Sox': 'CHW',
    'Los Angeles Angels': 'LAA',
    'Miami Marlins': 'MIA',
    'New York Mets': 'NYM',
    'Athletics': 'ATH',
    'St. Louis Cardinals': 'STL',
    'San Diego Padres': 'SDP',
    'Houston Astros': 'HOU'
}

def_rank['team'] = def_rank['team'].map(team_mapping)

In [9]:
#fixing the accentes on the names
def normalize_name(name):
    if not isinstance(name, str):
        return name
    return unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')

In [10]:
batter_team_2025['player'] = batter_team_2025['player'].apply(normalize_name)

In [11]:
#cleaning the # and *
def clean_name_symbols(name):
    if not isinstance(name, str):
        return name
    return name.replace('*', '').replace('#', '')

batter_team_2025['player'] = batter_team_2025['player'].apply(clean_name_symbols)

In [12]:
# multi_team_flags = ['2TM', '3TM', '4TM', '5TM']
# filtered_df_2 = batter_team_2025[~batter_team_2025['team'].isin(multi_team_flags)].copy()


# # Step 2: Keep only the LAST team listed for each player (i.e., current team)
# batter_team_2025 = filtered_df_2.drop_duplicates(subset='player', keep='last').reset_index(drop=True)

In [13]:
# Set API Key
headers = {
    'X-Api-Key': 'fac7e1a69f2baa47da395bde36dc869c'  # Ensure this is correct
}

# API URL
url = 'https://api.sportsgameodds.com/v2/events/'

next_cursor = None
event_data = []

# Fetch only Over/Under odds for rbi across all players
while True:
    try:
        response = requests.get(
            url,
            headers=headers,
            params={
                "sportID":"BASEBALL",
                "leagueID": "MLB",
                "oddIDs": "batting_RBI-PLAYER_ID-game-ou-over",
                "includeOpposingOddIDs": "true", 
                "oddsAvailable":"true",
                "cursor": next_cursor,
                "include":"teams",
                'started':"false",

            },
        )

        response.raise_for_status()  # Raise an error if the request fails

        data = response.json()
        event_data.extend(data.get("data", []))
        next_cursor = data.get("nextCursor")

        if not next_cursor:
            break

    except requests.exceptions.RequestException as e:
        print("Error fetching events:", e)
        break



In [14]:
event_data

[{'eventID': 'cDWGktUsQQXA4NRluc0P',
  'sportID': 'BASEBALL',
  'leagueID': 'MLB',
  'type': 'match',
  'teams': {'home': {'teamID': 'NEW_YORK_YANKEES_MLB',
    'names': {'long': 'New York Yankees', 'short': 'NYY', 'medium': 'Yankees'},
    'colors': {'primary': '#000066', 'primaryContrast': '#FFFFFF'},
    'statEntityID': 'home'},
   'away': {'teamID': 'NEW_YORK_METS_MLB',
    'names': {'long': 'New York Mets', 'short': 'NYM', 'medium': 'Mets'},
    'colors': {'primary': '#032D75', 'primaryContrast': '#FFFFFF'},
    'statEntityID': 'away'}},
  'odds': {'batting_RBI-AARON_JUDGE_1_MLB-game-ou-over': {'oddID': 'batting_RBI-AARON_JUDGE_1_MLB-game-ou-over',
    'opposingOddID': 'batting_RBI-AARON_JUDGE_1_MLB-game-ou-under',
    'marketName': 'Aaron Judge Runs Batted In Over/Under',
    'statID': 'batting_RBI',
    'statEntityID': 'AARON_JUDGE_1_MLB',
    'periodID': 'game',
    'betTypeID': 'ou',
    'sideID': 'over',
    'playerID': 'AARON_JUDGE_1_MLB',
    'started': False,
    'ended': 

In [15]:
# Initialize an empty list for structured data
odds_list = []

for event in event_data:
    event_id = event.get("eventID", "Unknown Event")
    teams = sorted(set(player_data.get("teamID") for player_data in event.get("players", {}).values() if player_data.get("teamID")))
    
    odds = event.get("odds", {})
    
    for odd_id, odd_object in odds.items():
        base_row = {
            "eventID": event_id,
            "teams": ", ".join(teams),
            "oddID": odd_object.get("oddID"),
            "opposingOddID": odd_object.get("opposingOddID"),
            "marketName": odd_object.get("marketName"),
            "playerID": odd_object.get("statEntityID"),
            "side": odd_object.get("sideID"),
            "line": odd_object.get("bookOverUnder", odd_object.get("fairOverUnder", "N/A")),
            "score": odd_object.get("score", "N/A"),
        }
        
        # Extract bookmaker-specific odds into separate rows
        by_bookmaker = odd_object.get("byBookmaker", {})
        
        for bookmaker, book_data in by_bookmaker.items():
            row = base_row.copy()
            row["bookmaker"] = bookmaker
            row["odds"] = book_data.get("odds", "N/A")
            row["overUnder"] = book_data.get("overUnder", "N/A")
            odds_list.append(row)

# Convert to Pandas DataFrame
df_odds = pd.DataFrame(odds_list)

# Print the DataFrame
df_odds.head(5)

,eventID,teams,oddID,opposingOddID,marketName,playerID,side,line,score,bookmaker,odds,overUnder
0,cDWGktUsQQXA4NRluc0P,"NEW_YORK_METS_MLB, NEW_YORK_YANKEES_MLB",batting_RBI-AARON_JUDGE_1_MLB-game-ou-over,batting_RBI-AARON_JUDGE_1_MLB-game-ou-under,Aaron Judge Runs Batted In Over/Under,AARON_JUDGE_1_MLB,over,0.5,N/A,draftkings,-105,0.5
1,cDWGktUsQQXA4NRluc0P,"NEW_YORK_METS_MLB, NEW_YORK_YANKEES_MLB",batting_RBI-AARON_JUDGE_1_MLB-game-ou-over,batting_RBI-AARON_JUDGE_1_MLB-game-ou-under,Aaron Judge Runs Batted In Over/Under,AARON_JUDGE_1_MLB,over,0.5,N/A,espnbet,+100,0.5
2,cDWGktUsQQXA4NRluc0P,"NEW_YORK_METS_MLB, NEW_YORK_YANKEES_MLB",batting_RBI-AARON_JUDGE_1_MLB-game-ou-over,batting_RBI-AARON_JUDGE_1_MLB-game-ou-under,Aaron Judge Runs Batted In Over/Under,AARON_JUDGE_1_MLB,over,0.5,N/A,hardrockbet,+100,0.5
3,cDWGktUsQQXA4NRluc0P,"NEW_YORK_METS_MLB, NEW_YORK_YANKEES_MLB",batting_RBI-AARON_JUDGE_1_MLB-game-ou-over,batting_RBI-AARON_JUDGE_1_MLB-game-ou-under,Aaron Judge Runs Batted In Over/Under,AARON_JUDGE_1_MLB,over,0.5,N/A,fliff,-105,0.5
4,cDWGktUsQQXA4NRluc0P,"NEW_YORK_METS_MLB, NEW_YORK_YANKEES_MLB",batting_RBI-AARON_JUDGE_1_MLB-game-ou-over,batting_RBI-AARON_JUDGE_1_MLB-game-ou-under,Aaron Judge Runs Batted In Over/Under,AARON_JUDGE_1_MLB,over,0.5,N/A,fanduel,-120,0.5


In [16]:
df_odds['bookmaker'].unique()

array(['draftkings', 'espnbet', 'hardrockbet', 'fliff', 'fanduel',
       'fanatics', 'ballybet', 'betrivers', 'betmgm', 'prizepicks'],
      dtype=object)

In [17]:
#getting the last 3 letters of the teams collumn to get MLB
df_odds['sport'] = df_odds['teams'].str[-3:]

In [18]:
# Split into two new columns
df_odds[['team_1', 'team_2']] = df_odds['teams'].str.split(', ', expand=True)
# Remove the "_MLB" suffix from both columns using .loc
df_odds.loc[:, 'team_1'] = df_odds['team_1'].str.replace('_MLB', '', regex=False)
df_odds.loc[:, 'team_2'] = df_odds['team_2'].str.replace('_MLB', '', regex=False)
# Drop the original 'teams' column
df_odds = df_odds.drop(columns=['teams'])

In [19]:
df_odds

,eventID,oddID,opposingOddID,marketName,playerID,side,line,score,bookmaker,odds,overUnder,sport,team_1,team_2
0,cDWGktUsQQXA4NRluc0P,batting_RBI-AARON_JUDGE_1_MLB-game-ou-over,batting_RBI-AARON_JUDGE_1_MLB-game-ou-under,Aaron Judge Runs Batted In Over/Under,AARON_JUDGE_1_MLB,over,0.5,N/A,draftkings,-105,0.5,MLB,NEW_YORK_METS,NEW_YORK_YANKEES
1,cDWGktUsQQXA4NRluc0P,batting_RBI-AARON_JUDGE_1_MLB-game-ou-over,batting_RBI-AARON_JUDGE_1_MLB-game-ou-under,Aaron Judge Runs Batted In Over/Under,AARON_JUDGE_1_MLB,over,0.5,N/A,espnbet,+100,0.5,MLB,NEW_YORK_METS,NEW_YORK_YANKEES
2,cDWGktUsQQXA4NRluc0P,batting_RBI-AARON_JUDGE_1_MLB-game-ou-over,batting_RBI-AARON_JUDGE_1_MLB-game-ou-under,Aaron Judge Runs Batted In Over/Under,AARON_JUDGE_1_MLB,over,0.5,N/A,hardrockbet,+100,0.5,MLB,NEW_YORK_METS,NEW_YORK_YANKEES
3,cDWGktUsQQXA4NRluc0P,batting_RBI-AARON_JUDGE_1_MLB-game-ou-over,batting_RBI-AARON_JUDGE_1_MLB-game-ou-under,Aaron Judge Runs Batted In Over/Under,AARON_JUDGE_1_MLB,over,0.5,N/A,fliff,-105,0.5,MLB,NEW_YORK_METS,NEW_YORK_YANKEES
4,cDWGktUsQQXA4NRluc0P,batting_RBI-AARON_JUDGE_1_MLB-game-ou-over,batting_RBI-AARON_JUDGE_1_MLB-game-ou-under,Aaron Judge Runs Batted In Over/Under,AARON_JUDGE_1_MLB,over,0.5,N/A,fanduel,-120,0.5,MLB,NEW_YORK_METS,NEW_YORK_YANKEES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,nF4FjrcX6JlEKRRvN7AT,batting_RBI-CHRIS_TAYLOR_1_MLB-game-ou-under,batting_RBI-CHRIS_TAYLOR_1_MLB-game-ou-over,Chris Taylor Runs Batted In Over/Under,CHRIS_TAYLOR_1_MLB,under,0.5,N/A,espnbet,-425,0.5,MLB,LOS_ANGELES_ANGELS,LOS_ANGELES_DODGERS
3676,nF4FjrcX6JlEKRRvN7AT,batting_RBI-CHRIS_TAYLOR_1_MLB-game-ou-under,batting_RBI-CHRIS_TAYLOR_1_MLB-game-ou-over,Chris Taylor Runs Batted In Over/Under,CHRIS_TAYLOR_1_MLB,under,0.5,N/A,draftkings,-390,0.5,MLB,LOS_ANGELES_ANGELS,LOS_ANGELES_DODGERS
3677,nF4FjrcX6JlEKRRvN7AT,batting_RBI-CHRIS_TAYLOR_1_MLB-game-ou-under,batting_RBI-CHRIS_TAYLOR_1_MLB-game-ou-over,Chris Taylor Runs Batted In Over/Under,CHRIS_TAYLOR_1_MLB,under,0.5,N/A,fliff,-400,0.5,MLB,LOS_ANGELES_ANGELS,LOS_ANGELES_DODGERS
3678,nF4FjrcX6JlEKRRvN7AT,batting_RBI-CHRIS_TAYLOR_1_MLB-game-ou-under,batting_RBI-CHRIS_TAYLOR_1_MLB-game-ou-over,Chris Taylor Runs Batted In Over/Under,CHRIS_TAYLOR_1_MLB,under,0.5,N/A,hardrockbet,-400,0.5,MLB,LOS_ANGELES_ANGELS,LOS_ANGELES_DODGERS


In [20]:
#extract the players name from marketName
df_odds['player'] = df_odds['marketName'].str.extract(r'^(.*?)\s*Runs')

In [21]:
# Define the mapping dictionary
team_mapping = {
    'ARIZONA_DIAMONDBACKS': 'ARI',
    'ATLANTA_BRAVES': 'ATL',
    'BALTIMORE_ORIOLES': 'BAL',
    'BOSTON_RED_SOX': 'BOS',
    'CHICAGO_CUBS': 'CHC',
    'CHICAGO_WHITE_SOX': 'CHW',
    'CINCINNATI_REDS': 'CIN',
    'CLEVELAND_GUARDIANS': 'CLE',
    'COLORADO_ROCKIES': 'COL',
    'DETROIT_TIGERS': 'DET',
    'HOUSTON_ASTROS': 'HOU',
    'KANSAS_CITY_ROYALS': 'KCR',
    'LOS_ANGELES_ANGELS': 'LAA',
    'LOS_ANGELES_DODGERS': 'LAD',
    'MIAMI_MARLINS': 'MIA',
    'MILWAUKEE_BREWERS': 'MIL',
    'MINNESOTA_TWINS': 'MIN',
    'NEW_YORK_METS': 'NYM',
    'NEW_YORK_YANKEES': 'NYY',
    'OAKLAND_ATHLETICS': 'ATH',
    'PHILADELPHIA_PHILLIES': 'PHI',
    'PITTSBURGH_PIRATES': 'PIT',
    'SAN_DIEGO_PADRES': 'SDP',
    'SAN_FRANCISCO_GIANTS': 'SFG',
    'SEATTLE_MARINERS': 'SEA',
    'STLOUIS_CARDINALS': 'STL',
    'TAMPA_BAY_RAYS': 'TBR',
    'TEXAS_RANGERS': 'TEX',
    'TORONTO_BLUE_JAYS': 'TOR',
    'WASHINGTON_NATIONALS': 'WSH'}


# Apply the mapping
df_odds.loc[:, 'team_1'] = df_odds['team_1'].map(team_mapping)
df_odds.loc[:, 'team_2'] = df_odds['team_2'].map(team_mapping)

In [22]:
#making the line collumn an integer
df_odds['overUnder'] = pd.to_numeric(df_odds['overUnder'], errors='coerce')

In [23]:
df_odds['bet']='RBI'

In [24]:
df_odds=df_odds[['player','side','overUnder','bookmaker','odds','sport','team_1','team_2','bet']]

In [25]:
# Merge on the 'player' column
df_odds = df_odds.merge(
    batter_team_2025[['player', 'team']],
    on='player',
    how='left'
)


In [26]:
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,team_1,team_2,bet,team
0,Aaron Judge,over,0.5,draftkings,-105,MLB,NYM,NYY,RBI,NYY
1,Aaron Judge,over,0.5,espnbet,+100,MLB,NYM,NYY,RBI,NYY
2,Aaron Judge,over,0.5,hardrockbet,+100,MLB,NYM,NYY,RBI,NYY
3,Aaron Judge,over,0.5,fliff,-105,MLB,NYM,NYY,RBI,NYY
4,Aaron Judge,over,0.5,fanduel,-120,MLB,NYM,NYY,RBI,NYY
...,...,...,...,...,...,...,...,...,...,...
3675,Chris Taylor,under,0.5,espnbet,-425,MLB,LAA,LAD,RBI,LAD
3676,Chris Taylor,under,0.5,draftkings,-390,MLB,LAA,LAD,RBI,LAD
3677,Chris Taylor,under,0.5,fliff,-400,MLB,LAA,LAD,RBI,LAD
3678,Chris Taylor,under,0.5,hardrockbet,-400,MLB,LAA,LAD,RBI,LAD


In [27]:
# Step 1: Identify the "odd one out" team to find the defensive team
df_odds['def_team'] = df_odds.apply(lambda row: row['team_2'] if row['team'] == row['team_1'] else row['team_1'], axis=1)

In [28]:
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,team_1,team_2,bet,team,def_team
0,Aaron Judge,over,0.5,draftkings,-105,MLB,NYM,NYY,RBI,NYY,NYM
1,Aaron Judge,over,0.5,espnbet,+100,MLB,NYM,NYY,RBI,NYY,NYM
2,Aaron Judge,over,0.5,hardrockbet,+100,MLB,NYM,NYY,RBI,NYY,NYM
3,Aaron Judge,over,0.5,fliff,-105,MLB,NYM,NYY,RBI,NYY,NYM
4,Aaron Judge,over,0.5,fanduel,-120,MLB,NYM,NYY,RBI,NYY,NYM
...,...,...,...,...,...,...,...,...,...,...,...
3675,Chris Taylor,under,0.5,espnbet,-425,MLB,LAA,LAD,RBI,LAD,LAA
3676,Chris Taylor,under,0.5,draftkings,-390,MLB,LAA,LAD,RBI,LAD,LAA
3677,Chris Taylor,under,0.5,fliff,-400,MLB,LAA,LAD,RBI,LAD,LAA
3678,Chris Taylor,under,0.5,hardrockbet,-400,MLB,LAA,LAD,RBI,LAD,LAA


In [29]:
#drop redundant collumns that are useless
df_odds = df_odds.drop(columns=['team_1', 'team_2'], errors='ignore')

In [30]:
# Rename the team column before merging (if needed)
def_rank = def_rank.rename(columns={"team": "def_team"})

In [31]:
# Merge on the 'player' column
df_odds = df_odds.merge(def_rank, on='def_team', how='left')
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,bet,team,def_team,Unnamed: 0,def_rank
0,Aaron Judge,over,0.5,draftkings,-105,MLB,RBI,NYY,NYM,0,1
1,Aaron Judge,over,0.5,espnbet,+100,MLB,RBI,NYY,NYM,0,1
2,Aaron Judge,over,0.5,hardrockbet,+100,MLB,RBI,NYY,NYM,0,1
3,Aaron Judge,over,0.5,fliff,-105,MLB,RBI,NYY,NYM,0,1
4,Aaron Judge,over,0.5,fanduel,-120,MLB,RBI,NYY,NYM,0,1
...,...,...,...,...,...,...,...,...,...,...,...
3675,Chris Taylor,under,0.5,espnbet,-425,MLB,RBI,LAD,LAA,25,25
3676,Chris Taylor,under,0.5,draftkings,-390,MLB,RBI,LAD,LAA,25,25
3677,Chris Taylor,under,0.5,fliff,-400,MLB,RBI,LAD,LAA,25,25
3678,Chris Taylor,under,0.5,hardrockbet,-400,MLB,RBI,LAD,LAA,25,25


In [32]:
#making game collumn
df_odds['game'] = df_odds['team'] + ' vs ' + df_odds['def_team']

In [33]:
df_odds2=df_odds.dropna()

In [34]:
df_odds3 = df_odds2.drop(columns=['team', 'def_team'])

In [35]:
df_odds3

,player,side,overUnder,bookmaker,odds,sport,bet,Unnamed: 0,def_rank,game
0,Aaron Judge,over,0.5,draftkings,-105,MLB,RBI,0,1,NYY vs NYM
1,Aaron Judge,over,0.5,espnbet,+100,MLB,RBI,0,1,NYY vs NYM
2,Aaron Judge,over,0.5,hardrockbet,+100,MLB,RBI,0,1,NYY vs NYM
3,Aaron Judge,over,0.5,fliff,-105,MLB,RBI,0,1,NYY vs NYM
4,Aaron Judge,over,0.5,fanduel,-120,MLB,RBI,0,1,NYY vs NYM
...,...,...,...,...,...,...,...,...,...,...
3675,Chris Taylor,under,0.5,espnbet,-425,MLB,RBI,25,25,LAD vs LAA
3676,Chris Taylor,under,0.5,draftkings,-390,MLB,RBI,25,25,LAD vs LAA
3677,Chris Taylor,under,0.5,fliff,-400,MLB,RBI,25,25,LAD vs LAA
3678,Chris Taylor,under,0.5,hardrockbet,-400,MLB,RBI,25,25,LAD vs LAA


In [36]:
overs = df_odds3[(df_odds3['side'].str.lower() == 'over') & (df_odds3['def_rank'] > 15)]
unders = df_odds3[(df_odds3['side'].str.lower() == 'under') & (df_odds3['def_rank'] < 16)]

In [37]:
overs = overs.merge(over_rbi_rate, on='player', how='left')
unders = unders.merge(under_rbi_rate, on='player', how='left')

In [38]:
overs_2=overs.dropna()
unders_2=unders.dropna()

In [39]:
#making odds collumn numeric
unders_2['odds'] = pd.to_numeric(unders_2['odds'], errors='coerce')
overs_2['odds'] = pd.to_numeric(overs_2['odds'], errors='coerce')

C:\Users\Joshua\AppData\Local\Temp\ipykernel_147632\2110421890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['odds'] = pd.to_numeric(unders_2['odds'], errors='coerce')
C:\Users\Joshua\AppData\Local\Temp\ipykernel_147632\2110421890.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['odds'] = pd.to_numeric(overs_2['odds'], errors='coerce')


In [40]:
#making decimal odds
unders_2['decimal_odds'] = unders_2['odds'].apply(
    lambda x: (x / 100) + 1 if x > 0 else (100 / abs(x)) + 1
)
overs_2['decimal_odds'] = overs_2['odds'].apply(
    lambda x: (x / 100) + 1 if x > 0 else (100 / abs(x)) + 1
)

C:\Users\Joshua\AppData\Local\Temp\ipykernel_147632\875580270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['decimal_odds'] = unders_2['odds'].apply(
C:\Users\Joshua\AppData\Local\Temp\ipykernel_147632\875580270.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['decimal_odds'] = overs_2['odds'].apply(


In [41]:
#making profit
overs_2['profit'] = (overs_2['decimal_odds'] * 100) - 100
unders_2['profit'] = (unders_2['decimal_odds'] * 100) - 100

C:\Users\Joshua\AppData\Local\Temp\ipykernel_147632\3093691362.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['profit'] = (overs_2['decimal_odds'] * 100) - 100
C:\Users\Joshua\AppData\Local\Temp\ipykernel_147632\3093691362.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['profit'] = (unders_2['decimal_odds'] * 100) - 100


In [42]:
def calculate_ev(row):
    if row['overUnder'] == 0.5:
        rate = row['1_RBI_rate_over']
    elif row['overUnder'] == 1.5:
        rate = row['2_RBI_rate_over']
    elif row['overUnder'] == 2.5:
        rate = row['3_RBI_rate_over']
    elif row['overUnder'] == 3.5:
        rate = row['4_RBI_rate_over']
    else:
        return None  # or np.nan, if you want to leave it blank for other lines

    # Expected Value formula
    return (rate * row['profit']) - ((1 - rate) * 100)

# Apply the formula to each row
overs_2['edge'] = overs_2.apply(calculate_ev, axis=1)

C:\Users\Joshua\AppData\Local\Temp\ipykernel_147632\882109118.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['edge'] = overs_2.apply(calculate_ev, axis=1)


In [43]:
#edge for unders
def calculate_under_ev(row):
    if row['overUnder'] == 0.5:
        rate = row['1_RBI_rate_under']
    elif row['overUnder'] == 1.5:
        rate = row['2_RBI_rate_under']
    elif row['overUnder'] == 2.5:
        rate = row['3_RBI_rate_under']
    elif row['overUnder'] == 3.5:
        rate = row['4_RBI_rate_under']
    else:
        return None  # or np.nan for unsupported lines

    return ((1 - rate) * row['profit']) - (rate * 100)

# Apply to each row
unders_2['edge'] = unders_2.apply(calculate_under_ev, axis=1)

C:\Users\Joshua\AppData\Local\Temp\ipykernel_147632\2277347561.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['edge'] = unders_2.apply(calculate_under_ev, axis=1)


In [44]:
overs_2 = overs_2[overs_2['edge'] > 20]

In [45]:
unders_2 = unders_2[unders_2['edge'] > 20]

In [46]:
#merging overs_2 on top of unders_2
final_df = pd.concat([overs_2, unders_2], ignore_index=True)

In [47]:
final_df=final_df[['player','side','overUnder','bookmaker','odds','sport','bet','edge','game']]

In [48]:
final_df['bet'] = final_df['side'].astype(str) + ' ' + final_df['overUnder'].astype(str) + ' ' + final_df['bet'].astype(str)


In [49]:
final_df = final_df.rename(columns={'bookmaker': 'book'})

In [50]:
final_df['edge'] = final_df['edge'] / 2

In [51]:
final_df['edge'] = final_df['edge'].round(1)

In [52]:
final_df=final_df[['book','sport','player','game','bet','odds','edge']]

In [53]:
final_df = final_df.sort_values(by='edge', ascending=False).reset_index(drop=True)


In [54]:
# #changing book names
# team_mapping_3 = {
#     'betonline': 'BetOnline',
#     'bovada': 'Bovada',
#     'fanatics': 'Fanatics',
#     'hardrockbet': 'Hard Rock Bet',
#     'sporttrade': 'Sporttrade',
#     'balleybet': 'Balley Bet',
#     'betmgm': 'BetMGM',
#     'draftkings': 'DraftKings',
#     'ceasers': 'Ceasers',
#     'fanduel': 'FanDuel',
#     'fliff': 'Fliff',
#     'espnbet': 'ESPNBET',
#     'betrivers': 'BetRivers'
# }

# final_df['book'] = final_df['book'].map(team_mapping_3)

In [55]:
#putting + in front of the ones that dont have it
final_df['odds'] = final_df['odds'].apply(lambda x: f"+{x}" if x > 0 else str(x))

final_df

,book,sport,player,game,bet,odds,edge
0,betrivers,MLB,Michael A. Taylor,CHW vs CHC,over 0.5 RBI,+320,104.3
1,ballybet,MLB,Michael A. Taylor,CHW vs CHC,over 0.5 RBI,+320,104.3
2,fanatics,MLB,Michael A. Taylor,CHW vs CHC,over 0.5 RBI,+300,97.0
3,fanduel,MLB,Michael A. Taylor,CHW vs CHC,over 0.5 RBI,+300,97.0
4,hardrockbet,MLB,Michael A. Taylor,CHW vs CHC,over 0.5 RBI,+300,97.0
...,...,...,...,...,...,...,...
329,espnbet,MLB,Jose Altuve,HOU vs TEX,under 0.5 RBI,-325,10.2
330,draftkings,MLB,Brent Rooker,ATH vs SFG,under 0.5 RBI,-230,10.2
331,fliff,MLB,Brent Rooker,ATH vs SFG,under 0.5 RBI,-230,10.2
332,espnbet,MLB,Austin Hedges,CLE vs CIN,under 0.5 RBI,-450,10.1


In [56]:
# # Convert DataFrame to JSON (Python list of dicts)
# data_json = json.loads(final_df.to_json(orient="records"))

# # Endpoint URL
# url = "https://pitstoppicks.com/wp-json/custom-api/v1/mlb_batter_total_RBI"

# # Make sure this is your FULL WordPress login email (not just 'josheskragmailcom')
# username = "josheskra@gmail.com"

# # This must be your app password EXACTLY as shown when created — include all spaces
# password = "BFgx z2iv nW9I voXn 1X51 fjRq"

# # Headers
# headers = {
#     "Content-Type": "application/json"
# }

# # POST request
# response = requests.post(
#     url,
#     headers=headers,
#     auth=HTTPBasicAuth(username, password),
#     json=data_json
# )

# # Debug
# print("Status Code:", response.status_code)
# print("Response Text:", response.text)

In [57]:
import json
import requests

# Convert your DataFrame to JSON
data_json = json.loads(final_df.to_json(orient="records"))

# Endpoint URL
url = "https://pitstoppicks.com/wp-json/custom-api/v1/mlb_batter_RBI"

# Headers
headers = {
    "Content-Type": "application/json"
}

# Send the POST request without authentication
response = requests.post(url, headers=headers, json=data_json)

# Debugging output
print("Status Code:", response.status_code)
print("Response Text:", response.text)


Status Code: 200
Response Text: {"message":"Data received","data":[{"book":"betrivers","sport":"MLB","player":"Michael A. Taylor","game":"CHW vs CHC","bet":"over 0.5 RBI","odds":"+320","edge":104.3},{"book":"ballybet","sport":"MLB","player":"Michael A. Taylor","game":"CHW vs CHC","bet":"over 0.5 RBI","odds":"+320","edge":104.3},{"book":"fanatics","sport":"MLB","player":"Michael A. Taylor","game":"CHW vs CHC","bet":"over 0.5 RBI","odds":"+300","edge":97},{"book":"fanduel","sport":"MLB","player":"Michael A. Taylor","game":"CHW vs CHC","bet":"over 0.5 RBI","odds":"+300","edge":97},{"book":"hardrockbet","sport":"MLB","player":"Michael A. Taylor","game":"CHW vs CHC","bet":"over 0.5 RBI","odds":"+300","edge":97},{"book":"draftkings","sport":"MLB","player":"Michael A. Taylor","game":"CHW vs CHC","bet":"over 0.5 RBI","odds":"+290","edge":93.3},{"book":"betmgm","sport":"MLB","player":"Michael A. Taylor","game":"CHW vs CHC","bet":"over 0.5 RBI","odds":"+290","edge":93.3},{"book":"betrivers","spo

In [58]:
# # Convert DataFrame to JSON (ensure it's a Python list, not a string)
# data_json = json.loads(final_df.to_json(orient="records"))  # ✅ Converts string to Python object

# # WordPress API URL
# url = "https://pitstoppicks.com/wp-json/custom-api/v1/mlb_batter_RBI"

# # Authentication
# # auth = HTTPBasicAuth("josheskragmailcom", "v!(d$mQZnAxSqek!1iAD$d!Q")
# auth = HTTPBasicAuth("josheskragmailcom", "UhiM WLwm dEOK T171 RUkT Sw2F")
# # Headers
# headers = {
#     "Content-Type": "application/json"
# }

# # Send POST request (requests will handle JSON encoding automatically)
# response = requests.post(url, headers=headers, auth=auth, json=data_json)  # ✅ FIX: Use json= instead of data=

# # Debugging Output
# print("Status Code:", response.status_code)
# print("Response Text:", response.text)

In [59]:
#YOU CAN DELETE ALL OF THIS THIS IS FOR WHEN I KEEP TRACK OF THE BETS LOCALLY

In [60]:
# Step 1: Sort to ensure reproducibility in tie cases
df_sorted = final_df.sort_values('edge')

# Step 2: Define a function to get the row with median edge for each group
def get_median_row(group):
    median_edge = group['edge'].median()
    # Get the row closest to the median (or first in case of exact match/ties)
    return group.iloc[(group['edge'] - median_edge).abs().argsort().iloc[0]]

# Step 3: Apply it to each group
df_filtered = df_sorted.groupby(['player', 'game', 'bet'], group_keys=False).apply(get_median_row)

# Step 4: Reset index if needed
df_filtered.reset_index(drop=True, inplace=True)
df_filtered

C:\Users\Joshua\AppData\Local\Temp\ipykernel_147632\1266780210.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtered = df_sorted.groupby(['player', 'game', 'bet'], group_keys=False).apply(get_median_row)


,book,sport,player,game,bet,odds,edge
0,hardrockbet,MLB,Addison Barger,TOR vs DET,under 0.5 RBI,-230,16.8
1,betmgm,MLB,Alec Bohm,PHI vs PIT,over 0.5 RBI,+170,15.2
2,betmgm,MLB,Andrew McCutchen,PIT vs PHI,under 0.5 RBI,-275,11.5
3,hardrockbet,MLB,Andrew Vaughn,CHW vs CHC,over 0.5 RBI,+150,48.3
4,hardrockbet,MLB,Anthony Santander,TOR vs DET,under 0.5 RBI,-180,19.8
5,draftkings,MLB,Austin Hays,CIN vs CLE,over 0.5 RBI,+120,35.8
6,draftkings,MLB,Austin Hedges,CLE vs CIN,under 0.5 RBI,-425,10.7
7,betmgm,MLB,Austin Wells,NYY vs NYM,under 0.5 RBI,-200,17.9
8,draftkings,MLB,Ben Rice,NYY vs NYM,under 0.5 RBI,-240,17.1
9,hardrockbet,MLB,Brent Rooker,ATH vs SFG,under 0.5 RBI,-220,11.0


In [61]:
# Make sure odds column is numeric
df_filtered['odds'] = pd.to_numeric(df_filtered['odds'], errors='coerce')
# Convert American odds to decimal odds
def american_to_decimal(odds):
    if pd.isna(odds):
        return None
    if odds > 0:
        return round((odds / 100) + 1, 3)
    else:
        return round((100 / abs(odds)) + 1, 3)

# Apply conversion
df_filtered['decimal_odds'] = df_filtered['odds'].apply(american_to_decimal)

# Check it out
print(df_filtered[['odds', 'decimal_odds']].head())

   odds  decimal_odds
0  -230         1.435
1   170         2.700
2  -275         1.364
3   150         2.500
4  -180         1.556


In [62]:
df_filtered['profit'] = (df_filtered['decimal_odds'] * 100) - 100
df_filtered

,book,sport,player,game,bet,odds,edge,decimal_odds,profit
0,hardrockbet,MLB,Addison Barger,TOR vs DET,under 0.5 RBI,-230,16.8,1.435,43.5
1,betmgm,MLB,Alec Bohm,PHI vs PIT,over 0.5 RBI,170,15.2,2.700,170.0
2,betmgm,MLB,Andrew McCutchen,PIT vs PHI,under 0.5 RBI,-275,11.5,1.364,36.4
3,hardrockbet,MLB,Andrew Vaughn,CHW vs CHC,over 0.5 RBI,150,48.3,2.500,150.0
4,hardrockbet,MLB,Anthony Santander,TOR vs DET,under 0.5 RBI,-180,19.8,1.556,55.6
5,draftkings,MLB,Austin Hays,CIN vs CLE,over 0.5 RBI,120,35.8,2.200,120.0
6,draftkings,MLB,Austin Hedges,CLE vs CIN,under 0.5 RBI,-425,10.7,1.235,23.5
7,betmgm,MLB,Austin Wells,NYY vs NYM,under 0.5 RBI,-200,17.9,1.500,50.0
8,draftkings,MLB,Ben Rice,NYY vs NYM,under 0.5 RBI,-240,17.1,1.417,41.7
9,hardrockbet,MLB,Brent Rooker,ATH vs SFG,under 0.5 RBI,-220,11.0,1.455,45.5


In [63]:
df_filtered.to_csv('median_rbi_bets.csv')